In [1]:
import pandas as pd
import random

# 항목 정의
genders = ["남성", "여성"]
foods = ["참치", "김치찌개", "햄버거", "비빔밥", "샐러드", "삼겹살", "라면", "스시", "과일", "샌드위치"]
lifestyles = [
    "건강 중시", "가성비 중시", "간편함 추구", "프리미엄 선호",
    "친환경 생활", "가족 중심", "여가 중시", "디지털 중심", "브랜드 충성"
]
usage_freq = ["월 10회 이상", "월 6~9회", "월 3~5회", "월 1~2회", "거의 안함"]
recency_options = ["3일 이내", "2주 이내", "한달 이내", "한달 이상"]
monetary_options = ["0~5만원", "5~10만원", "10~20만원", "20~30만원", "30만원 이상"]
purchase_channels = ["오프라인마트", "온라인몰", "모바일앱", "홈쇼핑"]

# 인구 비율 기반 나이대
age_distribution = {
    "20대": 0.17,
    "30대": 0.18,
    "40대": 0.20,
    "50대": 0.18,
    "60대": 0.14,
    "70대": 0.09,
    "80대 이상": 0.04
}

# 나이대별 가족구성
age_family_map = {
    "20대": ["1인 가구", "2인 부부", "부모동거", "자녀 1명"],
    "30대": ["2인 부부", "자녀 1명", "자녀 2명", "부모동거"],
    "40대": ["자녀 1명", "자녀 2명", "자녀 3명이상", "1인 가구", "2인 부부"],
    "50대": ["자녀 1명", "자녀 2명", "자녀 3명이상", "1인 가구"],
    "60대": ["자녀 1명", "자녀 2명", "자녀 3명이상", "1인 가구", "2인 부부"],
    "70대": ["1인 가구", "2인 부부"],
    "80대 이상": ["1인 가구", "2인 부부"]
}

# 나이대별 직업군
age_job_map = {
    "20대": ["학생", "회사원", "프리랜서", "전업주부"],
    "30대": ["회사원", "프리랜서", "자영업", "공무원", "전업주부"],
    "40대": ["회사원", "자영업", "공무원", "프리랜서", "전업주부"],
    "50대": ["회사원", "자영업", "공무원", "프리랜서", "전업주부"],
    "60대": ["자영업", "공무원", "은퇴", "전업주부"],
    "70대": ["자영업", "은퇴"],
    "80대 이상": ["은퇴"]
}

# 매핑 함수들
def choose_job(age, gender, family):
    possible_jobs = age_job_map[age].copy()
    if "전업주부" in possible_jobs:
        if gender == "남성" or (gender == "여성" and family == "1인 가구"):
            possible_jobs.remove("전업주부")
    return random.choice(possible_jobs) if possible_jobs else "기타"

def map_recency_by_age_family(age, family):
    if age == "20대":
        return random.choice(["3일 이내", "2주 이내"])
    elif age == "30대":
        if "자녀" in family:
            return random.choice(["2주 이내", "한달 이내"])
        else:
            return "2주 이내"
    elif age == "40대":
        return random.choice(["2주 이내", "한달 이내"])
    elif age == "50대":
        return random.choice(["한달 이내", "한달 이상"])
    elif age == "60대":
        return random.choice(["한달 이내", "한달 이상"])
    else:
        return "한달 이상"

def map_monetary_by_age_family(age, family):
    if age == "20대":
        return random.choice(["0~5만원", "5~10만원"])
    elif age == "30대":
        if "자녀" in family:
            return random.choice(["10~20만원", "20~30만원"])
        else:
            return random.choice(["5~10만원", "10~20만원"])
    elif age == "40대":
        if "자녀 2명" in family or "자녀 3명이상" in family:
            return random.choice(["20~30만원", "30만원 이상"])
        else:
            return "10~20만원"
    elif age == "50대":
        return random.choice(["10~20만원", "20~30만원"])
    elif age == "60대":
        return random.choice(["5~10만원", "10~20만원"])
    else:
        return random.choice(["0~5만원", "5~10만원"])

def map_purchase_channel(age, family, job):
    if age in ["20대", "30대"] and family == "1인 가구":
        return random.choice(["모바일앱", "온라인몰"])
    elif "자녀" in family:
        return "오프라인마트"
    elif age in ["60대", "70대", "80대 이상"]:
        if job == "은퇴":
            return random.choice(["홈쇼핑", "오프라인마트"])
        else:
            return "오프라인마트"
    else:
        return random.choice(["오프라인마트", "온라인몰"])

# 데이터 생성
total_rows = 1000
age_counts = {age: round(total_rows * pct) for age, pct in age_distribution.items()}

while sum(age_counts.values()) != total_rows:
    diff = total_rows - sum(age_counts.values())
    max_group = max(age_counts, key=age_counts.get)
    age_counts[max_group] += diff

data = []
row_id = 1

for age, count in age_counts.items():
    for _ in range(count):
        gender = random.choice(genders)
        family = random.choice(age_family_map[age])
        job = choose_job(age, gender, family)
        freq = random.choice(usage_freq)
        rec = map_recency_by_age_family(age, family)
        mon = map_monetary_by_age_family(age, family)
        channel = map_purchase_channel(age, family, job)

        row = {
            "응답자ID": f"{row_id:03}",
            "성별": gender,
            "나이대": age,
            "가족구성": family,
            "직업": job,
            "좋아하는음식": ", ".join(random.sample(foods, k=random.randint(1, 3))),
            "라이프스타일": ", ".join(random.sample(lifestyles, k=random.randint(1, 2))),
            "동원제품이용횟수": freq,
            "Recency": rec,
            "Monetary": mon,
            "구매 채널": channel
        }
        data.append(row)
        row_id += 1

# 저장
df = pd.DataFrame(data)
df.to_csv("페르소나_test.csv", index=False, encoding="utf-8-sig")

In [7]:
import json

# 항목 정의
persona_categories = {
    "나이": ["20대", "30대", "40대", "50대", "60대이상"],
    "성별": ["남", "여"],
    "가족구성": ["1인가구", "부모동거", "부부", "자녀1명", "자녀2명 이상"],
    "고객취향": [
        "통조림/즉석/면류", "생수/음료/커피", "과자/떡/베이커리",
        "냉장/냉동/간편식", "유제품", "건강식품"
    ],
    "고객가치(RFM)": ["VIP", "우수고객", "잠재우수고객", "신규고객", "잠재이탈고객", "이탈/휴면고객"],
    "라이프스타일": ["트랜드추종", "가격민감", "브랜드선호", "건강중시"]
}

# 단일 선택 함수
def get_user_input(category, options):
    print(f"\n{category} 선택:")
    for i, option in enumerate(options, 1):
        print(f"  {i}. {option}")
    while True:
        choice = input(f"번호를 입력하세요 (1~{len(options)}): ").strip()
        if choice.isdigit() and 1 <= int(choice) <= len(options):
            return options[int(choice) - 1]
        print("유효한 번호를 입력해주세요.")

# 복수 선택 함수 (고객취향은 max_select=3, 라이프스타일은 제한 없음)
def get_multiple_choices(category, options, max_select=None):
    print(f"\n{category} 선택", end="")
    if max_select:
        print(f" (최대 {max_select}개까지 선택, 쉼표로 구분):")
    else:
        print(" (원하는 만큼 선택, 쉼표로 구분):")
    for i, option in enumerate(options, 1):
        print(f"  {i}. {option}")
    while True:
        choice_str = input(f"번호 입력 (예: 1,3,4): ").strip()
        indices = [s.strip() for s in choice_str.split(",") if s.strip()]
        if all(s.isdigit() and 1 <= int(s) <= len(options) for s in indices):
            selected = [options[int(i)-1] for i in indices]
            selected = list(dict.fromkeys(selected))
            if len(selected) >= 1 and (max_select is None or len(selected) <= max_select):
                return selected
        print("올바른 번호 형식 또는 개수를 입력해주세요.")

# 페르소나 구성
def build_persona():
    persona = {}
    for category, options in persona_categories.items():
        if category == "고객취향":
            persona[category] = get_multiple_choices(category, options, max_select=3)
        elif category == "라이프스타일":
            persona[category] = get_multiple_choices(category, options)
        else:
            persona[category] = get_user_input(category, options)
    return persona

# 자연어 프롬프트 생성
def format_persona_prompt(p):
    return (
        f"당신은 {p['나이']} {p['성별']}성이며, {p['가족구성']}입니다. "
        f"{p['고객가치(RFM)']}이며, "
        f"{' / '.join(p['고객취향'])}을(를) 선호하고, "
        f"{'/'.join(p['라이프스타일'])}한 라이프스타일을 가지고 있습니다."
    )

# 실행
print("페르소나를 선택해 주세요!")
persona = build_persona()
query = format_persona_prompt(persona)

print("\n생성된 쿼리:")
print(query)

페르소나를 선택해 주세요!

나이 선택:
  1. 20대
  2. 30대
  3. 40대
  4. 50대
  5. 60대이상
번호를 입력하세요 (1~5): 3

성별 선택:
  1. 남
  2. 여
번호를 입력하세요 (1~2): 2

가족구성 선택:
  1. 1인가구
  2. 부모동거
  3. 부부
  4. 자녀1명
  5. 자녀2명이상
번호를 입력하세요 (1~5): 5

고객취향 선택 (최대 3개까지 선택, 쉼표로 구분):
  1. 통조림/즉석/면류
  2. 생수/음료/커피
  3. 과자/떡/베이커리
  4. 냉장/냉동/간편식
  5. 유제품
  6. 건강식품
번호 입력 (예: 1,3,4): 2,4,6

고객가치(RFM) 선택:
  1. VIP
  2. 우수고객
  3. 잠재우수고객
  4. 신규고객
  5. 잠재이탈고객
  6. 이탈/휴면고객
번호를 입력하세요 (1~6): 3

라이프스타일 선택 (원하는 만큼 선택, 쉼표로 구분):
  1. 트랜드추종
  2. 가격민감
  3. 브랜드선호
  4. 건강중시
번호 입력 (예: 1,3,4): 2,4

생성된 쿼리:
당신은 40대 여성이며, 자녀2명이상입니다. 잠재우수고객이며, 생수/음료/커피 / 냉장/냉동/간편식 / 건강식품을(를) 선호하고, 가격민감 / 건강중시한 라이프스타일을 가지고 있습니다.


In [8]:
print(query)

당신은 40대 여성이며, 자녀2명이상입니다. 잠재우수고객이며, 생수/음료/커피 / 냉장/냉동/간편식 / 건강식품을(를) 선호하고, 가격민감 / 건강중시한 라이프스타일을 가지고 있습니다.


In [19]:
import pandas as pd
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_upstage import UpstageEmbeddings
from dotenv import load_dotenv
import re
import os

# CSV 파일 로드
df = pd.read_csv("페르소나_test.csv")

# 자연어 문장 변환 함수
def row_to_sentence(row):
    return (
        f"이 사람은 {row['나이대']} {row['성별']}이며, 가족구성은 {row['가족구성']}입니다. "
        f"{row['직업']}으로 일하고 있으며, 좋아하는 음식은 {row['좋아하는음식']}입니다. "
        f"라이프스타일은 {라이프스타일}이며, 일상에서 이를 중시합니다. "
        f"동원 제품은 최근 {row['Recency']}에 {row['동원제품이용횟수']} 정도 사용하였고, {row['Monetary']} 정도를 소비했습니다. "
        f"이를 종합하면 고객가치(RFM) 기준으로 '{rfm}'으로 분류됩니다."
    )


# 정규표현식 기반으로 항목 추출
def extract_attributes(text):
    # 나이대 정규화
    age_match = re.search(r"(20대|30대|40대|50대|60대|70대|80대 이상|60대 이상)", text)
    age = age_match.group(1) if age_match else None
    if age in ["60대", "70대", "80대 이상", "60대 이상"]:
        age = "60대이상"

    # 성별 정규화
    gender_match = re.search(r"(남성|여성)", text)
    gender = gender_match.group(1) if gender_match else None

    # 가족구성 정규화
    family_match = re.search(r"(1인가구|부모동거|부부|자녀1명|자녀2명|자녀3명 이상|자녀2명 이상)", text)
    if family_match:
        family = family_match.group(1)
        if family in ["자녀2명", "자녀3명 이상"]:
            family = "자녀2명 이상"
    else:
        family = None

    return age, gender, family

# 쿼리에서 조건 추출
target_age, target_gender, target_family = extract_attributes(query)

# 유효성 체크
if not all([target_age, target_gender, target_family]):
    print("쿼리에서 필요한 조건(나이대, 성별, 가족구성)을 모두 추출할 수 없습니다.")
else:
    print(f"추출된 조건 → 나이대: {target_age}, 성별: {target_gender}, 가족구성: {target_family}")
    
    # 나이대 필터링
    if target_age == "60대이상":
        age_filter = df["나이대"].isin(["60대", "70대", "80대 이상"])
    else:
        age_filter = df["나이대"] == target_age
    
    # 가족구성 필터링
    if target_family == "자녀2명 이상":
        family_filter = df["가족구성"].isin(["자녀 2명", "자녀 3명이상"])
    else:
        family_filter = df["가족구성"] == target_family

    # 조건 필터링
    filtered_df = df[
        age_filter &
        (df["성별"] == target_gender) &
        family_filter
    ]

    if filtered_df.empty:
        print("조건에 맞는 페르소나가 없습니다.")
    else:
        # 문서 생성
        documents = filtered_df.apply(lambda row: Document(page_content=row_to_sentence(row)), axis=1).tolist()

        # 임베딩 초기화
        load_dotenv()
        embedding = UpstageEmbeddings(model="embedding-query")

        # 벡터 저장소 생성 (메모리 기반)
        vectorstore = Chroma.from_documents(documents, embedding=embedding)

        # 유사도 검색
        results_with_score = vectorstore.similarity_search_with_score(query, k=3)

        print("\n유사한 페르소나 (유사도 포함):")
        for i, (doc, score) in enumerate(results_with_score, 1):
            similarity = 1 - score
            print(f"\n[Top {i}] (유사도: {similarity:.4f})")
            print(doc.page_content)

추출된 조건 → 나이대: 40대, 성별: 여성, 가족구성: 자녀2명 이상


NameError: name '라이프스타일' is not defined